## Imports + Ordnerstruktur

In [ ]:
import os
import time
import random
import glob

import pandas as pd
import geopandas as gpd
import osmnx as ox
import requests

import folium
from folium.plugins import HeatMap

# -------------------------
# Projektordner (alles an einem Ort)
# -------------------------
PROJECT_DIR = "scs_osm_project"   # <- kannst du umbenennen
CSV_PATH = "islands.csv"          # <- dein CSV-Dateiname (Semikolon)

CACHE_DIR  = os.path.join(PROJECT_DIR, "cache_gpkg")
MASTER_DIR = os.path.join(PROJECT_DIR, "master_gpkg")
HTML_DIR   = os.path.join(PROJECT_DIR, "heatmaps_html")

for d in [CACHE_DIR, MASTER_DIR, HTML_DIR]:
    os.makedirs(d, exist_ok=True)

print("Folders:")
print("CACHE :", CACHE_DIR)
print("MASTER:", MASTER_DIR)
print("HTML  :", HTML_DIR)


Folders:
CACHE: scs_osm_project\cache_gpkg
MASTER: scs_osm_project\master_gpkg
HTML : scs_osm_project\heatmaps_html


## CSV laden

In [ ]:
df = pd.read_csv(CSV_PATH, sep=";", engine="python")

needed = ["id", "island", "island_group", "country", "north", "south", "west", "east"]
missing = [c for c in needed if c not in df.columns]
if missing:
    raise ValueError(f"Fehlende Spalten in CSV: {missing}\nGefundene Spalten: {list(df.columns)}")

df = df[needed].copy()

# Zahlen robust parsen
for c in ["north", "south", "west", "east"]:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Zeilen ohne vollständige bbox ignorieren (dein Wunsch)
mask_ok = df[["north", "south", "west", "east"]].notna().all(axis=1)
skipped = df.loc[~mask_ok, ["id", "island"]].copy()
df = df.loc[mask_ok].copy()

print("Rows used (with bbox):", len(df))
print("Rows skipped (missing bbox):", len(skipped))
if len(skipped) > 0:
    display(skipped.head(20))

# OSMnx bbox order: (west, south, east, north)
df["bbox"] = list(zip(df["west"], df["south"], df["east"], df["north"]))

df.head()


Rows used (with bbox): 57
Rows skipped (missing bbox): 13


id            island
8    9   Drummond Island
9   10    Duncan Islands
12  13      Money Island
13  14      North Island
15  16  Observation Bank
16  17     Pattle Island
17  18     Quanfu Island
18  19     Robert Island
19  20      South Island
20  21        South Sand
21  22       Tree Island
23  24         West Sand
25  26     Yagong Island

id            island island_group country   north  south    west    east  \
0   1    Cuarteron Reef      Spratly   China   8.890   8.84  112.80  112.87   
1   2  Fiery Cross Reef      Spratly   China   9.580   9.53  112.86  112.92   
2   3       Gaven Reefs      Spratly   China  10.220  10.19  114.21  114.24   
3   4       Hughes Reef      Spratly   China   9.927   9.90  114.48  114.51   
4   5      Johnson Reef      Spratly   China   9.750   9.69  114.26  114.30   

                             bbox  
0     (112.8, 8.84, 112.87, 8.89)  
1    (112.86, 9.53, 112.92, 9.58)  
2  (114.21, 10.19, 114.24, 10.22)  
3    (114.48, 9.9, 114.51, 9.927)  
4     (114.26, 9.69, 114.3, 9.75)

## Insel-Auswahl

In [ ]:
TEST_ISLAND = "Fiery Cross Reef"

df_one = df[df["island"] == TEST_ISLAND].copy()
if df_one.empty:
    raise ValueError(f"'{TEST_ISLAND}' nicht gefunden. Prüfe Schreibweise in df['island'].")

row = df_one.iloc[0]

island_id = int(row["id"])
island_name = row["island"]
country = row["country"]
island_group = row["island_group"]
bbox = row["bbox"]

print("Selected island:")
display(df_one[["id","island","country","island_group","bbox"]])


Islands to process: 57

=== 1 | Cuarteron Reef (China) bbox=(112.8, 8.84, 112.87, 8.89) ===
  runways: exists=False
  helipads: exists=True
  helipads: downloaded 3 in 61.1s
  helipads: saved -> 001__Cuarteron_Reef__helipads.gpkg
  towers: exists=True
  towers: downloaded 5 in 61.1s
  towers: saved -> 001__Cuarteron_Reef__towers.gpkg
  marinas: exists=True


## Queries + Overpass Einstellungen

In [ ]:
# Wir verwenden "schöne" Typ-Namen für später (helipads, runways, towers...)
# -> das sind auch die Dateinamen.
QUERIES = {
    "runways": {"aeroway": "runway"},
    "helipads": {"aeroway": ["helipad", "heliport"]},
    "towers": {"man_made": "tower"},
    "marinas": {"leisure": "marina"},
    "ferry_terminals": {"amenity": "ferry_terminal"},
}

# OSMnx/Overpass Setup
ox.settings.use_cache = True
ox.settings.log_console = True
ox.settings.overpass_settings = "[out:json][timeout:180]"  # ohne ';'

# Mirror (oft schneller)
ox.settings.overpass_url = "https://overpass.kumi.systems/api/interpreter"
# Alternativen:
# ox.settings.overpass_url = "https://overpass.nchc.org.tw/api/interpreter"
# ox.settings.overpass_url = "https://overpass.openstreetmap.ru/api/interpreter"

# Rate-Limit (Overpass freundlich)
BASE_SLEEP = 5
JITTER = 2

def safe_sleep():
    time.sleep(BASE_SLEEP + random.uniform(0, JITTER))

print("Overpass URL:", ox.settings.overpass_url)
print("Queries:", list(QUERIES.keys()))


Overpass URL: https://overpass.kumi.systems/api/interpreter
Queries: ['runways', 'helipads', 'towers', 'marinas', 'ferry_terminals']


## Mini “Exists”-Check (optional, beschleunigt)

In [5]:
def bbox_to_overpass_str(bbox):
    # bbox: (west,south,east,north) -> overpass: (south,west,north,east)
    west, south, east, north = bbox
    return f"({south},{west},{north},{east})"

def overpass_exists(bbox, tags, timeout=90):
    """Schneller Check: liefert True, wenn >= 1 Element existiert."""
    bbox_str = bbox_to_overpass_str(bbox)

    # Falls tags Listen enthalten (helipad/heliport), bauen wir mehrere Selektoren
    selectors = []
    for k, v in tags.items():
        if isinstance(v, (list, tuple, set)):
            for vv in v:
                selectors.append((k, vv))
        else:
            selectors.append((k, v))

    parts = []
    for k, v in selectors:
        parts.append(f'node["{k}"="{v}"]{bbox_str};')
        parts.append(f'way["{k}"="{v}"]{bbox_str};')
        parts.append(f'relation["{k}"="{v}"]{bbox_str};')

    query = f"""
    [out:json][timeout:{timeout}];
    (
      {''.join(parts)}
    );
    out ids qt 1;
    """

    r = requests.post(ox.settings.overpass_url, data={"data": query}, timeout=timeout + 30)
    r.raise_for_status()
    data = r.json()
    return len(data.get("elements", [])) > 0


## Download + Cache als GeoPackage (pro Insel × Typ)

In [6]:
def slug(s: str) -> str:
    """macht Dateinamen sicher (keine Leerzeichen/Sonderzeichen)."""
    return "".join(ch if ch.isalnum() else "_" for ch in str(s)).strip("_")

def cache_path(island_id, island_name, layer_key):
    return os.path.join(CACHE_DIR, f"{island_id:03d}__{slug(island_name)}__{layer_key}.gpkg")

def fetch_features_bbox(bbox, tags):
    return ox.features_from_bbox(bbox=bbox, tags=tags).reset_index()

def download_island_to_cache(island_id, island_name, country, island_group, bbox, overwrite=False):
    """
    Lädt alle QUERIES für eine Insel und speichert sie als einzelne GPKGs in CACHE_DIR.
    Simpel: 1x exists-check; wenn exists=False -> skip; wenn exists=True -> download.
    """
    print(f"\n=== {island_id} | {island_name} ({country}) bbox={bbox} ===")

    for layer_key, tags in QUERIES.items():
        out_path = cache_path(island_id, island_name, layer_key)

        # 1) Cache benutzen, wenn vorhanden
        if (not overwrite) and os.path.exists(out_path):
            try:
                g_cached = gpd.read_file(out_path)
                print(f"  {layer_key}: cached ({len(g_cached)}) -> {os.path.basename(out_path)}")
                safe_sleep()
                continue
            except Exception:
                # wenn kaputt: einfach neu schreiben
                pass

        # 2) Exists-check (einmal)
        try:
            exists = overpass_exists(bbox, tags, timeout=90)
        except Exception as e:
            # 1 Retry bei echtem Serverproblem
            print(f"  {layer_key}: exists-check error -> retry once | {repr(e)}")
            safe_sleep()
            exists = overpass_exists(bbox, tags, timeout=120)

        print(f"  {layer_key}: exists={exists}")

        if not exists:
            # Kein Retry nötig, wenn es einfach nichts gibt
            safe_sleep()
            continue

        # 3) Download (1 Versuch reicht meist; wenn du willst, kannst du 2 machen)
        try:
            t0 = time.time()
            g = fetch_features_bbox(bbox, tags)
            dt = time.time() - t0
            print(f"  {layer_key}: downloaded {len(g)} in {dt:.1f}s")

            # Metadaten hinzufügen (damit später Master/Heatmap einfach ist)
            g["src_island_id"] = island_id
            g["src_island"] = island_name
            g["src_country"] = country
            g["src_group"] = island_group
            g["src_layer"] = layer_key

            # Speichern
            g.to_file(out_path, driver="GPKG")
            print(f"  {layer_key}: saved -> {os.path.basename(out_path)}")
        except Exception as e:
            print(f"  {layer_key}: download error (skipped) -> {repr(e)}")

        safe_sleep()


## Download starten

In [7]:
download_island_to_cache(
    island_id=island_id,
    island_name=island_name,
    country=country,
    island_group=island_group,
    bbox=bbox,
    overwrite=False
)



=== 2 | Fiery Cross Reef (China) bbox=(112.86, 9.53, 112.92, 9.58) ===
  runways: exists-check error -> retry once | JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
  runways: exists=True
  runways: downloaded 1 in 0.2s
  runways: saved -> 002__Fiery_Cross_Reef__runways.gpkg
  helipads: exists=True
  helipads: downloaded 5 in 0.1s
  helipads: saved -> 002__Fiery_Cross_Reef__helipads.gpkg
  towers: exists=True
  towers: downloaded 5 in 0.1s
  towers: saved -> 002__Fiery_Cross_Reef__towers.gpkg
  marinas: exists=True
  marinas: downloaded 1 in 0.1s
  marinas: saved -> 002__Fiery_Cross_Reef__marinas.gpkg
  ferry_terminals: exists=False


## Master pro Typ bauen (viel besser als hunderte Einzeldateien lesen)

In [ ]:
def build_master_global():
    """
    Liest ALLE Cache-GPKGs aus CACHE_DIR
    und schreibt pro Typ eine Master-GPKG nach MASTER_DIR.
    """
    files = sorted(glob.glob(os.path.join(CACHE_DIR, "*.gpkg")))
    print("Total cache files:", len(files))
    if not files:
        print("No cache files found.")
        return

    # pro Typ sammeln
    buckets = {k: [] for k in QUERIES.keys()}

    for fp in files:
        base = os.path.splitext(os.path.basename(fp))[0]
        # erwartet: 002__Fiery_Cross_Reef__helipads
        layer_key = base.split("__")[-1]

        if layer_key not in buckets:
            continue

        g = gpd.read_file(fp)
        if len(g) == 0:
            continue

        # falls src_layer fehlt, aus layer_key setzen
        if "src_layer" not in g.columns:
            g["src_layer"] = layer_key

        buckets[layer_key].append(g)

    # schreiben
    for layer_key, parts in buckets.items():
        out_master = os.path.join(MASTER_DIR, f"master__{layer_key}.gpkg")

        if not parts:
            print(f"Master {layer_key}: no data")
            continue

        g_all = gpd.GeoDataFrame(pd.concat(parts, ignore_index=True), geometry="geometry", crs=parts[0].crs)

        # einfache Dedup-Option (wenn osmid existiert)
        if "osmid" in g_all.columns:
            before = len(g_all)
            g_all = g_all.drop_duplicates(subset=["osmid", "src_layer"], keep="first")
            print(f"Master {layer_key}: dedup {before} -> {len(g_all)}")

        g_all.to_file(out_master, driver="GPKG")
        print(f"Master {layer_key}: saved {os.path.basename(out_master)} ({len(g_all)})")


Cache files found: 4
Master runways: saved -> master__runways.gpkg (1)
Master helipads: saved -> master__helipads.gpkg (5)
Master towers: saved -> master__towers.gpkg (5)
Master marinas: saved -> master__marinas.gpkg (1)
Master ferry_terminals: (no data)


In [ ]:
build_master_global()


## Heatmaps pro Typ aus Master-Dateien bilden

In [9]:
def to_points(gdf):
    """
    Für Heatmaps brauchen wir Punkte (lat/lon).
    - Points bleiben Points
    - Linien/Polygone -> centroid
    """
    g = gdf[gdf.geometry.notna()].copy()
    # centroid geht nur in projizierten CRS "sauber", aber für kleine Inseln reicht es hier als Näherung.
    g["point_geom"] = g.geometry.apply(lambda geom: geom if geom.geom_type == "Point" else geom.centroid)
    g["lon"] = g["point_geom"].x
    g["lat"] = g["point_geom"].y
    return g.dropna(subset=["lat", "lon"])

def make_heatmap_from_points(points_df, out_html, zoom_start=12):
    if len(points_df) == 0:
        print("No points ->", out_html)
        return None

    center = [points_df["lat"].mean(), points_df["lon"].mean()]
    m = folium.Map(location=center, zoom_start=zoom_start, tiles="CartoDB positron")

    HeatMap(
        data=points_df[["lat", "lon"]].values.tolist(),
        radius=18, blur=14, min_opacity=0.25
    ).add_to(m)

    m.save(out_html)
    print("Saved:", out_html)
    return m

# Heatmaps pro Typ erzeugen
for layer_key in QUERIES.keys():
    master_path = os.path.join(MASTER_DIR, f"master__{layer_key}.gpkg")
    out_html = os.path.join(HTML_DIR, f"heatmap__{layer_key}.html")

    if not os.path.exists(master_path):
        print("Missing master:", master_path)
        continue

    g = gpd.read_file(master_path)
    pts = to_points(g)

    # Zoom kannst du anpassen:
    make_heatmap_from_points(pts, out_html, zoom_start=12)


Saved: scs_osm_project\heatmaps_html\heatmap__runways.html
Saved: scs_osm_project\heatmaps_html\heatmap__helipads.html
Saved: scs_osm_project\heatmaps_html\heatmap__towers.html
Saved: scs_osm_project\heatmaps_html\heatmap__marinas.html
Missing master: scs_osm_project\master_gpkg\master__ferry_terminals.gpkg
